## Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("Libraries imported successfully!")

## Load and Explore Sales Data

In [ ]:
# Load data
df = pd.read_csv('../data/sales_historical.csv')

# Display basic information
print(f"Dataset Shape: {df.shape}")
print(f"\nColumn Data Types:")
print(df.dtypes)
print(f"\nFirst 5 rows:")
print(df.head())

## Data Quality Assessment

In [ ]:
# Check for missing values
print("Missing Values:")
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Missing_Percentage': missing_pct
})
print(missing_df[missing_df['Missing_Count'] > 0])

# Statistical Summary
print("\nStatistical Summary:")
print(df.describe())

## Data Cleaning and Preprocessing

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Convert Date to datetime
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# Handle missing values in Unit_Price
# Fill with category median
df_clean['Unit_Price'] = df_clean.groupby('Category')['Unit_Price'].transform(
    lambda x: x.fillna(x.median())
)

# Remove any remaining nulls
df_clean = df_clean.dropna()

# Remove duplicates
initial_rows = len(df_clean)
df_clean = df_clean.drop_duplicates()
removed_duplicates = initial_rows - len(df_clean)

print(f"Duplicates removed: {removed_duplicates}")
print(f"Final dataset shape: {df_clean.shape}")
print(f"\nMissing values after cleaning:")
print(df_clean.isnull().sum())

## Outlier Detection and Handling

In [ ]:
# Identify outliers using IQR method for Sales_Amount
Q1 = df_clean['Sales_Amount'].quantile(0.25)
Q3 = df_clean['Sales_Amount'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_clean[
    (df_clean['Sales_Amount'] < lower_bound) | 
    (df_clean['Sales_Amount'] > upper_bound)
]

print(f"Outliers detected: {len(outliers)} records ({(len(outliers)/len(df_clean)*100):.2f}%)")
print(f"\nOutlier Statistics:")
print(f"Lower Bound: ${lower_bound:.2f}")
print(f"Upper Bound: ${upper_bound:.2f}")
print(f"\nOutlier Sales Amounts:")
print(outliers['Sales_Amount'].describe())

# Keep outliers as they represent legitimate high-value transactions
print("\nNote: Outliers retained as they represent legitimate transactions")

## Exploratory Data Analysis

In [ ]:
# Aggregate sales by date for time series analysis
daily_sales = df_clean.groupby('Date').agg({
    'Sales_Amount': ['sum', 'mean', 'count'],
    'Quantity': 'sum'
}).reset_index()

daily_sales.columns = ['Date', 'Total_Sales', 'Avg_Transaction', 'Transaction_Count', 'Total_Quantity']

print("Daily Sales Summary:")
print(daily_sales.head(10))
print(f"\nDate Range: {daily_sales['Date'].min()} to {daily_sales['Date'].max()}")
print(f"Days in dataset: {len(daily_sales)}")

## Sales Trends Over Time

In [ ]:
# Plot daily sales trend
fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(daily_sales['Date'], daily_sales['Total_Sales'], 
        label='Daily Sales', linewidth=1, alpha=0.7, color='steelblue')

# Add 30-day moving average
ma_30 = daily_sales['Total_Sales'].rolling(window=30).mean()
ax.plot(daily_sales['Date'], ma_30, 
        label='30-Day Moving Average', linewidth=2.5, color='darkorange')

ax.set_xlabel('Date', fontsize=12, fontweight='bold')
ax.set_ylabel('Sales ($)', fontsize=12, fontweight='bold')
ax.set_title('Daily Sales Trend with 30-Day Moving Average', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Trend Analysis:")
print(f"Average Daily Sales: ${daily_sales['Total_Sales'].mean():.2f}")
print(f"Min Daily Sales: ${daily_sales['Total_Sales'].min():.2f}")
print(f"Max Daily Sales: ${daily_sales['Total_Sales'].max():.2f}")
print(f"Std Dev: ${daily_sales['Total_Sales'].std():.2f}")

## Monthly Aggregation and Seasonal Patterns

In [ ]:
# Aggregate to monthly level
df_clean['Year_Month'] = df_clean['Date'].dt.to_period('M')

monthly_sales = df_clean.groupby('Year_Month').agg({
    'Sales_Amount': ['sum', 'mean', 'count'],
    'Quantity': 'sum'
}).reset_index()

monthly_sales.columns = ['Year_Month', 'Total_Sales', 'Avg_Transaction', 'Transaction_Count', 'Total_Quantity']
monthly_sales['Year_Month'] = monthly_sales['Year_Month'].astype(str)

print("Monthly Sales Summary:")
print(monthly_sales)

# Plot monthly sales
fig, ax = plt.subplots(figsize=(14, 6))

ax.bar(range(len(monthly_sales)), monthly_sales['Total_Sales'], 
       color='steelblue', alpha=0.8, edgecolor='navy')
ax.plot(range(len(monthly_sales)), monthly_sales['Total_Sales'], 
        color='red', marker='o', linewidth=2, markersize=6, label='Trend')

ax.set_xlabel('Month', fontsize=12, fontweight='bold')
ax.set_ylabel('Sales ($)', fontsize=12, fontweight='bold')
ax.set_title('Monthly Sales Trend', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(monthly_sales)))
ax.set_xticklabels(monthly_sales['Year_Month'], rotation=45, ha='right')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Sales by Category

In [ ]:
# Sales by Category
category_sales = df_clean.groupby('Category')['Sales_Amount'].agg(['sum', 'mean', 'count']).sort_values('sum', ascending=False)
category_sales.columns = ['Total_Sales', 'Avg_Sales', 'Transaction_Count']

print("Sales by Category:")
print(category_sales)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart
colors = plt.cm.Set3(np.linspace(0, 1, len(category_sales)))
ax1.bar(category_sales.index, category_sales['Total_Sales'], color=colors, edgecolor='black')
ax1.set_title('Total Sales by Category', fontsize=12, fontweight='bold')
ax1.set_ylabel('Sales ($)', fontsize=11)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3, axis='y')

# Pie chart
ax2.pie(category_sales['Total_Sales'], labels=category_sales.index, autopct='%1.1f%%',
        colors=colors, startangle=90)
ax2.set_title('Sales Distribution by Category', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## Sales by Store and Region

In [ ]:
# Sales by Store
store_sales = df_clean.groupby('Store')['Sales_Amount'].agg(['sum', 'mean', 'count']).sort_values('sum', ascending=False)
store_sales.columns = ['Total_Sales', 'Avg_Sales', 'Transaction_Count']

print("Sales by Store:")
print(store_sales)

# Sales by Region
region_sales = df_clean.groupby('Region')['Sales_Amount'].agg(['sum', 'mean', 'count']).sort_values('sum', ascending=False)
region_sales.columns = ['Total_Sales', 'Avg_Sales', 'Transaction_Count']

print("\nSales by Region:")
print(region_sales)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Store sales
colors1 = plt.cm.Set2(np.linspace(0, 1, len(store_sales)))
ax1.barh(store_sales.index, store_sales['Total_Sales'], color=colors1, edgecolor='black')
ax1.set_title('Total Sales by Store', fontsize=12, fontweight='bold')
ax1.set_xlabel('Sales ($)', fontsize=11)
ax1.grid(True, alpha=0.3, axis='x')

# Region sales
colors2 = plt.cm.Pastel1(np.linspace(0, 1, len(region_sales)))
ax2.barh(region_sales.index, region_sales['Total_Sales'], color=colors2, edgecolor='black')
ax2.set_title('Total Sales by Region', fontsize=12, fontweight='bold')
ax2.set_xlabel('Sales ($)', fontsize=11)
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## Save Cleaned Data

In [ ]:
# Save cleaned data
df_clean_export = df_clean.copy()
df_clean_export = df_clean_export.drop('Year_Month', axis=1)
df_clean_export.to_csv('../data/sales_cleaned.csv', index=False)

# Save daily aggregated data
daily_sales.to_csv('../data/sales_daily.csv', index=False)

# Save monthly aggregated data
monthly_sales.to_csv('../data/sales_monthly.csv', index=False)

print("✓ sales_cleaned.csv")
print("✓ sales_daily.csv")
print("✓ sales_monthly.csv")
print("\nAll cleaned data files saved successfully!")

## Summary Statistics

In [ ]:
print("="*60)
print("DATA CLEANING SUMMARY")
print("="*60)
print(f"\nOriginal Records: {len(df):,}")
print(f"Cleaned Records: {len(df_clean):,}")
print(f"Records Removed: {len(df) - len(df_clean):,}")
print(f"\nDate Range: {df_clean['Date'].min().date()} to {df_clean['Date'].max().date()}")
print(f"Duration: {(df_clean['Date'].max() - df_clean['Date'].min()).days} days")
print(f"\nTotal Sales: ${df_clean['Sales_Amount'].sum():,.2f}")
print(f"Average Daily Sales: ${daily_sales['Total_Sales'].mean():,.2f}")
print(f"\nStores: {df_clean['Store'].nunique()}")
print(f"Categories: {df_clean['Category'].nunique()}")
print(f"Regions: {df_clean['Region'].nunique()}")
print(f"\nTotal Transactions: {len(df_clean):,}")
print(f"Total Units Sold: {df_clean['Quantity'].sum():,}")
print("="*60)